In [15]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
import nbimporter
from fetch_data_from_yfinance import data_puller
import util_data_cleaner
import warnings
warnings.filterwarnings("ignore")

In [2]:
def detect_frequency(date_df):
    date_diffs = date_df.diff().dropna()
    median_diff = date_diffs.median()
    
    daily_threshold = pd.Timedelta(days=1.5)
    weekly_threshold = pd.Timedelta(weeks=1.5)
    
    # Determine frequency based on the median difference
    if median_diff <= daily_threshold:
        return('1d')
    elif median_diff <= weekly_threshold:
        return('1wk')
    else:
        return('1mo')

In [28]:
def adding_incremental_data():
    
    existing_data = pd.read_csv(r'C:\Users\40100171\OneDrive - Anheuser-Busch InBev\Desktop\Stock_market_strategies\data\raw\stock_data_raw.csv.gz', compression='gzip')
    existing_data = util_data_cleaner.raw_data_cleaner(existing_data)
    stock_universe = existing_data['Stock Name'].unique().tolist()
    dates = pd.to_datetime(existing_data.loc[existing_data['Stock Name']==stock_universe[0],'Date'])
    
    start_date = dates.max() + timedelta(days=1) 
    end_date = date.today()
    interval = detect_frequency(dates)
    inter_df = []
    
    if(start_date != end_date):
        for i in stock_universe:
            df = data_puller(i, start_date, end_date, interval)
            inter_df.append(df)
        final_data = pd.concat(inter_df)
        final_data = pd.concat([existing_data, final_data])
        final_data.reset_index(inplace=True)
        final_data.to_csv(r'C:\Users\40100171\OneDrive - Anheuser-Busch InBev\Desktop\Stock_market_strategies\data\raw\stock_data_raw_try.csv.gz', index=False, compression='gzip')

In [29]:
adding_incremental_data()


1 Failed download:
['SICALLOG.NS']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2024-08-20 00:00:00 -> 2024-08-23)')


In [32]:
check = pd.read_csv(r'C:\Users\40100171\OneDrive - Anheuser-Busch InBev\Desktop\Stock_market_strategies\data\raw\stock_data_raw_try.csv.gz', compression='gzip')

In [34]:
check

,index,Date,Open,High,Low,Close,Adj Close,Volume,Stock Name
0,0,2020-01-01,1387.957031,1396.277466,1376.527954,1380.276611,1356.042725,7002234.0,reliance.NS
1,1,2020-01-02,1382.471069,1408.941040,1382.471069,1403.775024,1379.128662,8855158.0,reliance.NS
2,2,2020-01-03,1401.671997,1409.581055,1392.528687,1405.466553,1380.790527,10492349.0,reliance.NS
3,3,2020-01-06,1389.785767,1397.008911,1369.670410,1372.870605,1348.766846,12259588.0,reliance.NS
4,4,2020-01-07,1388.871338,1403.043579,1383.842529,1393.991699,1369.517090,8341811.0,reliance.NS
...,...,...,...,...,...,...,...,...,...
1898621,2024-08-21 00:00:00,NaN,113.650002,113.650002,113.650002,113.650002,113.650002,0.0,girresorts.NS
1898622,2024-08-22 00:00:00,NaN,113.650002,113.650002,113.650002,113.650002,113.650002,0.0,girresorts.NS
1898623,2024-08-20 00:00:00,NaN,64.599998,64.599998,64.599998,64.599998,64.599998,0.0,3rdrock.NS
1898624,2024-08-21 00:00:00,NaN,64.599998,64.599998,64.599998,64.599998,64.599998,0.0,3rdrock.NS
